In [40]:
from Classical_Building_Blocks import knapsack_solver_classical
from Quantum_Building_Blocks import knapsack_solver_quantum

ModuleNotFoundError: No module named 'qiskit'

# KNAPSACK PROBLEM
    As part of the capacitated hetrogenous vehicle routing problem

# Table of Contents

1. [Introduction](#Introduction)
1. [Data Loading](#Data-Loading)
1. [Classical Method](#Classical-Method)
    * [Description](#Classical-Description)
    * [Results](#classical-Resaults)
1. [Quantum Method](#Classical)
    * [QAOA](#QAOA)
    * [Other Algorithm](#Other-Algorithms)
    * [docplex](#Docplex)
    * [Results](#Quantum-Resaults)
    * [Quantum-Circuit](#Quantum-Resaults)
1. [Benchmarking](#Benchmarking)
    * [Quantum Advantages](#Quantum-Advantages)
    * [Algorithms Benchmarking](#Algorithms-benchmarking)
1. [Conclusions](#Conclusions)
1. [Future Work](#Future-Work)

# Introduction
The Knapsack Problem is a classic computational optimization challenge. This problem poses the question: given a list of items, each with a specific weight and value, and a knapsack with a limited carrying capacity, what combination of items should be chosen to maximize the total value without surpassing the knapsack's weight limit?

In the context of this notebook, we will be applying our exploration of the Knapsack Problem to a more complex scenario - the Capacitated Vehicle Routing Problem (CVRP) as discussed in the paper accessible via the following link: https://arxiv.org/pdf/2110.06799.pdf.

The C-HVRP (capacitated hetrogenous vehicle routing problem) involves not just deciding what items to pack in a knapsack (or, in this case, cargo in a vehicle), but also determining the best routes for a fleet of vehicles, each with its own capacity limitation, delivering goods to a set of customers.

To tackle these problems, we will employ a variety of optimization algorithms, including a brute force approach, the Quantum Approximate Optimization Algorithm (QAOA), and several others. These methods will be thoroughly compared and benchmarked, providing an insight into their respective strengths and weaknesses. We aim to highlight the different results and computational efficiencies these algorithms offer when used to solve such complex optimization problems.

The Hamiltonian which we need to simulate and maximized, as presented in the papre, are shown are:


$$ Ha = A (1 - \sum\limits_1^W(Z_n))^2 + A(\sum\limits_1^W(nZ_n) - \sum\limits_1^N(W_iX_i))^2 $$

$$Hb = -B \sum\limits_i^N(c_i x_i)$$



$$C(X') = M(W_{max} - \sum_{i=0}^{n-1} x_{i}w_{i} - ֿֿ\sum(2j * y[j]))^2 - \sum_{i}^{n-1} x_{i}v_{i}$$

We can extract out of it two expressions: Quadratic & Linear, which we then solve and optimize using various quantum optimization methods

Evantually, when it will be runn together with the HVRP solution, this is the hamiltonian to be parsed, as it containt terms that regards the HVRP problem
see [link to HVRP main page]
$$
Ha = A * \sum\limits_{v}\left(\left( \sum\limits_{k=0}^{M^v-1} (2^k z_k^v + (Q^v + 1 - 2^{M^v}  ) z_M^v\right) -  \sum\limits_{a,i}q_i y_{ia}^v\right)^2
$$



# Data Loading


In [ ]:
data_file = 'data/small_data'
max_knapsack_weight = 25

# Classical Method

## Description

Solving the knapsack as in a classical 0/1 Knapsack Problem using Dynamic Programming, specifically a tabulation (or bottom-up) approach is presented here.
The function knapsack_solver_classical(W, wt, v, N) solves the problem by constructing a two-dimensional table K[][] in bottom-up fashion.

In this function, `W` is the maximum weight the knapsack can carry, `wt` is the list of weights for each item, `v` is the list of values for each item, and `N` is the number of items. 
`K[i][w]` represents the maximum value that can be obtained with a knapsack capacity `w` using the first `i` items.
The function iterates over every item and for each one, it considers all possible maximum weights from 0 to `W`. If the weight of the current item is less than or equal to the current capacity `w`, it determines whether to include the item in the optimal subset or not by comparing the total value with or without the current item (`v[j-1] + K[j-1][w-wt[j-1]]` vs `K[j-1][w])`, choosing the maximum of the two. If the weight of the current item is greater than w, it simply carries over the maximum value found for the previous item at the same capacity. The function returns the maximum possible value for the given knapsack capacity and the entire table `K`.

In the next part of the code, the specific items included in the optimal solution are determined by backtracking through the K table, starting from `K[N][W]`. If the value at `K[j][w]` is different from the value at `K[j-1][w]`, it indicates that the jth item was included in the optimal subset. The code keeps track of these chosen items, their corresponding weights, and values, and continuously reduces the remaining capacity of the knapsack by the weight of the chosen item. This process continues until no items are left (`j = 0`) or the knapsack has no remaining capacity (`w = 0`). The final solution is presented in a detailed manner, providing the IDs, values, and weights of the chosen items, along with the total number of items, the total value, and the total weight of the items in the knapsack.

The time complexity of this method `O(N*W)`, where `N` is the number of items and `W`  is the maximum weight of the knapsack. We can solve this problem using an exact solution approach within a reasonable time since the number of combinations is limited, but when the number of items becomes huge, it will be impractical to deal with by using a exact solution approach.

## Results

In [ ]:

chosen_objects, chosen_values, chosen_weights, max_weigth, total_objects, total_value, chosen_weights_sum = knapsack_solver_classical(data_file, max_knapsack_weight)

print("your knapsack will contain:")
for i in range(len(chosen_objects)):
    print("Object:", chosen_objects[i], "- Value:", chosen_values[i], "- Weight:", chosen_weights[i])
print("The maximum weight the knapsack can have is:", max_weigth, "\nAnd according to the data provided, we can put", total_objects, "objects inside it\nwith a total value of", total_value, "\nand a total weight of", chosen_weights_sum)

# Quantum Method

In [ ]:
knapsack_solver_quantum(data_file, max_knapsack_weight)

Now, we can define a vector x as x = [Z_1, Z_2, ..., Z_W]^T, where the superscript T denotes the transpose of the vector.

Revisiting the expression:

$$A(n^2(Z_1^2 + Z_1Z_2 + ... + Z_1Z_W + Z_2Z_1 + Z_2^2 + ... + Z_2Z_W + ... + Z_WZ_1 + Z_WZ_2 + ... + Z_W^2) - 2n(\sum\limits_1^W(Z_n))(sum(W_iX_i)) + \sum\limits_1^N\sum\limits_1^N w_{ij})$$

We can rewrite it as:

$$A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))(sum(W_iX_i)) + \sum\limits_1^N\sum\limits_1^N w_{ij})$$

Here, we substituted the matrix Q as defined earlier. Now, let's continue simplifying:

$$A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))(\sum\limits_1^N(W_iX_i)) + \sum\limits_1^N\sum\limits_1^N w_{ij})

= A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))(\sum\limits_1^N(W_iX_i)) + \sum\limits_1^N\sum\limits_1^N (W_iX_i)(W_jX_j))$$

Now, let's define a vector g of size N as $$ g = \sum\limits_1^N(W_iX_i).$$

The expression becomes:

$$A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))g + g^TQg)$$

Finally, let's define a constant c as $$ c = g^TQg.$$

The expression simplifies to:

$$ A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))g + c) $$

Therefore, the simplified expression is $$ A(n^2(x^TQx) - 2n(\sum\limits_1^W(Z_n))g + c) $$ , where Q is an N x N matrix, x and g are vectors of appropriate sizes, and c is a constant.